In [1]:
import os
Root = "/content/drive/MyDrive/speech-emotion-recognition-ravdess-data"
os.chdir(Root)

In [3]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [4]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [5]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [6]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [7]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [8]:
x_train

array([[-5.17969666e+02,  3.71328316e+01, -1.57047663e+01, ...,
         1.15746159e-04,  8.88544528e-05,  4.30482396e-05],
       [-5.31415405e+02,  2.07455730e+01, -9.86208344e+00, ...,
         1.53095287e-03,  1.27632730e-03,  5.98607236e-04],
       [-6.76233093e+02,  4.29739914e+01, -1.94877994e+00, ...,
         1.25673105e-05,  7.65429468e-06,  6.76830359e-06],
       ...,
       [-5.00130554e+02,  1.76183472e+01, -1.49308767e+01, ...,
         6.52547518e-04,  4.70803556e-04,  2.16973320e-04],
       [-7.49890991e+02,  3.04202042e+01, -5.73750198e-01, ...,
         8.31350098e-06,  1.02366748e-05,  7.03076239e-06],
       [-5.43855835e+02,  5.56097260e+01, -7.47326183e+00, ...,
         3.50014860e-04,  1.91711981e-04,  1.59476578e-04]])

In [9]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(168, 56)


In [10]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [11]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [12]:
#Train the model
model.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:612: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  "Got `batch_size` less than 1 or larger than "


MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [13]:
#Predict for the test set
y_pred=model.predict(x_test)

In [14]:
y_pred

array(['fearful', 'fearful', 'disgust', 'happy', 'fearful', 'fearful',
       'calm', 'happy', 'fearful', 'disgust', 'disgust', 'fearful',
       'calm', 'fearful', 'disgust', 'happy', 'fearful', 'happy',
       'disgust', 'disgust', 'happy', 'happy', 'happy', 'calm', 'disgust',
       'fearful', 'disgust', 'disgust', 'fearful', 'happy', 'happy',
       'happy', 'calm', 'happy', 'calm', 'fearful', 'fearful', 'disgust',
       'calm', 'fearful', 'calm', 'disgust', 'calm', 'calm', 'fearful',
       'calm', 'calm', 'disgust', 'fearful', 'calm', 'happy', 'disgust',
       'disgust', 'disgust', 'calm', 'calm'], dtype='<U7')

In [15]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 78.57%


In [16]:
from sklearn.metrics import accuracy_score, f1_score

In [17]:
f1_score(y_test, y_pred,average=None)

array([0.88      , 0.8       , 0.83870968, 0.61538462])

In [18]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,fearful,fearful
1,happy,fearful
2,disgust,disgust
3,happy,happy
4,fearful,fearful
5,fearful,fearful
6,disgust,calm
7,fearful,happy
8,fearful,fearful
9,happy,disgust
